In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive

In [ ]:
# set up code detectron2 để chạy predict dc cái boundingbox 

!python -m pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'


import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)


# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import ColorMode

register_coco_instances("pill_segmentation", {}, "/content/drive/MyDrive/train_pill/label.json", "/content/drive/MyDrive/train_pill/pill_images")
dataset_dicts = DatasetCatalog.get("pill_segmentation")
pill_metadata = MetadataCatalog.get("pill_segmentation")

# set up đơn giản để chạy
cfg = get_cfg()
cfg.merge_from_file(
     "./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"
)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # 1 classes (pill)
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.95   # set the testing threshold for this model
cfg.DATASETS.TEST = ("pill_segmentation2", )
predictor = DefaultPredictor(cfg)



# nhớ thay đổi tên đường dẫn tấm hình
im = cv2.imread('/content/VAIPE_P_0_4.jpg')
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=pill_metadata, 
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
# read file label json của btc

f = open('/content/VAIPE_P_0_4.json')
data = json.load(f)
boxes = []
for b in data:
  box = {}
  box["box"] = torch.tensor(np.array([[b['x'],b['y'], b['w'] + b['x'] , b['h'] + b['y']]]),dtype=torch.float)
  box["label"] =b['label']
  boxes.append(box)
pred_boxes = list(outputs['instances'].to("cpu").get_fields()["pred_boxes"])



# tính iou giữ pred và từng gt rồi lấy iou lớn nhất
result =[]
for pred in pred_boxes:
  temp = []
  for gt in boxes:
    dic = {}
    dic['iou'] = bops.box_iou(pred.unsqueeze(0), gt['box'])
    dic['label'] = gt['label']
    temp.append(dic)

  newlist = sorted(temp, key=lambda d: d['iou'], reverse=True) 
  result.append({'box' : pred , 'label' : newlist[0]['label']})

# in ra kết quả
print(result)